In [1]:
from datetime import datetime as dt
import plotly.express as px
import dash                                     
from dash import dcc, html, Input, Output
import requests, json
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# preparing DataFrame to work with / подготавливаем данные для работы
# get names of assets / получаем названия валют
assets = requests.get('http://api.coincap.io/v2/assets').json()
assets_id = []
assets_symbol = []
for i in assets['data']:
    assets_id.append(i['id'])
    assets_symbol.append(i['symbol'])
assets_name = dict(zip(assets_id, assets_symbol))

# create DataFrame / создаем таблицу данных
df = pd.DataFrame()
for key,value in tqdm(assets_name.items()):
    url = 'http://api.coincap.io/v2/assets/' + str(key) + '/history?interval=d1'
    json = requests.get(url).json()
    dff = pd.DataFrame(json['data'])
    dff['asset'] = assets_name.get(key)
    df = pd.concat([df, dff])

# fix types / исправляем типы данных
df['date'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S.%f").dt.date
df['date'] = pd.to_datetime(df['date'])
df['priceUsd'] = df['priceUsd'].astype('float')
df = df.drop(columns='time')

df.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,priceUsd,date,asset
0,35535.843759,2022-01-23,BTC
1,35184.823228,2022-01-24,BTC
2,36585.761042,2022-01-25,BTC
3,37593.286290,2022-01-26,BTC
4,36443.727781,2022-01-27,BTC


In [3]:
# load style / загружаем стиль
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# create dashboard / отрисовываем дашборд
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
                html.Div([
                    html.Div([
                        html.Label('Select an asset'),
                        dcc.Dropdown(
                                    options=assets_name,
                                    value='bitcoin',
                                    multi=False,
                                    id='my_dropdown',
                        ),
                        html.Div([
                            html.Div([
                                html.Label('Date from'),
                                dcc.DatePickerSingle(  
                                    min_date_allowed=df['date'].dt.date.min(),
                                    max_date_allowed=df['date'].dt.date.max(),
                                    date=df['date'].dt.date.min(),
                                    display_format='DD MM YYYY',
                                    id='my_date_picker_range_start'
                                ),
                            ], className = 'six columns'),
                            html.Div([
                                html.Label('Date to'),
                                dcc.DatePickerSingle(  
                                    min_date_allowed=df['date'].dt.date.min(),
                                    max_date_allowed=df['date'].dt.date.max(),
                                    date=df['date'].dt.date.max(),
                                    display_format='DD MM YYYY',
                                    id='my_date_picker_range_end'
                                ),
                            ], className = 'six columns'),
                        ], className = 'row'),
                    ], className = 'four columns'),
                    html.Div([
                        dcc.Graph(id='fig')
                    ], className = 'eight columns'),
                ], className = 'row'), 
            ])

@app.callback(
    Output('fig', 'figure'),
    Input('my_date_picker_range_start', 'date'),
    Input('my_date_picker_range_end', 'date'),
    Input('my_dropdown', 'value')
)
def update_output(start_date, end_date, value):
    mask = (df['date'] > start_date) & (df['date'] <= end_date) & (df['asset'] == assets_name.get(value)) 
    data = df.loc[mask]
    fig = px.bar(data, x="date", y="priceUsd")
    
    fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    title=assets_name.get(value),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)')
    
    return fig

# run dashboard / запускаем
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jan/2023 15:44:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:36] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2023 15:44:37] "POST /_dash-update-component HTTP/1.1" 200 -
